In [1]:
from PIL import Image
import os
path='chest_xray_lower_dim200x200'
old_path='chest_xray_lower_dim/'
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
import cv2
import glob
files=glob.glob('chest_xray_lower_dim/*.jpeg')
dataset=[]
labels=[]
import PIL
from PIL import Image
import cv2
for file in files:
    img=Image.open(file)
    img=img.convert(mode='L')#too much information without reason x-rays are BlackWhite
    img=img.resize((200,200))#we try to stop curse of dimensionality
    img.save('chest_xray_lower_dim200x200/'+file[len(old_path):])
print('Done')

Done


In [1]:
def create_dataset(directory):
    #before work
    import cv2
    import glob
    files=glob.glob(directory+'/*.jpeg')
    dataset=[]
    labels=[]
    for file in files:
        img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
        dataset.append(img)
        #write your code
        if "virus" in file:
            labels.append("0")
        elif "bacteria" in file:
            labels.append("1")
        else:
            labels.append("2")
    
    from numpy import asarray
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset,labels

In [2]:
dataset,labels=create_dataset('chest_xray_lower_dim2')
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

In [10]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint


def train(epochs_num,batch_len,conv_layers,neur_layers):
    X_train,X_test,Y_train,Y_test=train_test_split(
    dataset, labels, test_size=0.2, random_state=42)
    X_train=X_train.reshape(-1,100,100,1)
    X_test=X_test.reshape(-1,100,100,1)
    Y_train = keras.utils.to_categorical(Y_train,3)
    Y_test = keras.utils.to_categorical(Y_test,3)
    
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation="relu",kernel_regularizer=regularizers.L2(0.01),kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=inputs*30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation="softmax",kernel_regularizer=regularizers.L2(0.01))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.2,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [11]:
train(60,50,3,3)

Epoch 1/60
75/75 - 43s - loss: 1.1199 - accuracy: 0.4756 - val_loss: 1.0960 - val_accuracy: 0.4813 - 43s/epoch - 573ms/step
Epoch 2/60
75/75 - 40s - loss: 1.0866 - accuracy: 0.4788 - val_loss: 1.0768 - val_accuracy: 0.4813 - 40s/epoch - 538ms/step
Epoch 3/60
75/75 - 38s - loss: 1.0201 - accuracy: 0.4788 - val_loss: 0.9077 - val_accuracy: 0.4813 - 38s/epoch - 509ms/step
Epoch 4/60
75/75 - 38s - loss: 0.9115 - accuracy: 0.5343 - val_loss: 0.8596 - val_accuracy: 0.6617 - 38s/epoch - 512ms/step
Epoch 5/60
75/75 - 38s - loss: 0.8721 - accuracy: 0.6667 - val_loss: 0.8262 - val_accuracy: 0.6585 - 38s/epoch - 509ms/step
Epoch 6/60
75/75 - 38s - loss: 0.8156 - accuracy: 0.6843 - val_loss: 0.7875 - val_accuracy: 0.6660 - 38s/epoch - 511ms/step
Epoch 7/60
75/75 - 38s - loss: 0.7530 - accuracy: 0.6966 - val_loss: 0.7629 - val_accuracy: 0.6531 - 38s/epoch - 513ms/step
Epoch 8/60
75/75 - 38s - loss: 0.7183 - accuracy: 0.6976 - val_loss: 0.7118 - val_accuracy: 0.6766 - 38s/epoch - 509ms/step
Epoch 9/

In [19]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint


def train(epochs_num,batch_len,conv_layers,neur_layers):
    X_train,X_test,Y_train,Y_test=train_test_split(
    dataset, labels, test_size=0.2, random_state=42)
    X_train=X_train.reshape(-1,100,100,1)
    X_test=X_test.reshape(-1,100,100,1)
    Y_train = keras.utils.to_categorical(Y_train,3)
    Y_test = keras.utils.to_categorical(Y_test,3)
    
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=2560
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation="relu",kernel_regularizer=regularizers.L2(0.01),kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation="softmax",kernel_regularizer=regularizers.L2(0.01),kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.2,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [20]:
train(60,50,3,3)

Epoch 1/60
75/75 - 41s - loss: 1.2125 - accuracy: 0.4716 - val_loss: 1.1348 - val_accuracy: 0.4813 - 41s/epoch - 544ms/step
Epoch 2/60
75/75 - 38s - loss: 1.0983 - accuracy: 0.4788 - val_loss: 1.0215 - val_accuracy: 0.4813 - 38s/epoch - 506ms/step
Epoch 3/60
75/75 - 38s - loss: 0.9143 - accuracy: 0.5941 - val_loss: 0.7908 - val_accuracy: 0.6596 - 38s/epoch - 510ms/step
Epoch 4/60
75/75 - 38s - loss: 0.7856 - accuracy: 0.6832 - val_loss: 0.7397 - val_accuracy: 0.6809 - 38s/epoch - 507ms/step
Epoch 5/60
75/75 - 38s - loss: 0.7282 - accuracy: 0.7000 - val_loss: 0.8243 - val_accuracy: 0.6820 - 38s/epoch - 512ms/step
Epoch 6/60
75/75 - 39s - loss: 0.7177 - accuracy: 0.7043 - val_loss: 0.6805 - val_accuracy: 0.6958 - 39s/epoch - 524ms/step
Epoch 7/60
75/75 - 39s - loss: 0.6751 - accuracy: 0.7222 - val_loss: 0.7046 - val_accuracy: 0.7044 - 39s/epoch - 519ms/step
Epoch 8/60
75/75 - 39s - loss: 0.6794 - accuracy: 0.7315 - val_loss: 0.6587 - val_accuracy: 0.7279 - 39s/epoch - 524ms/step
Epoch 9/

In [21]:
train(60,200,3,3)

Epoch 1/60
19/19 - 48s - loss: 1.2990 - accuracy: 0.4628 - val_loss: 1.2223 - val_accuracy: 0.4813 - 48s/epoch - 3s/step
Epoch 2/60
19/19 - 38s - loss: 1.2069 - accuracy: 0.4761 - val_loss: 1.1772 - val_accuracy: 0.4813 - 38s/epoch - 2s/step
Epoch 3/60
19/19 - 39s - loss: 1.1663 - accuracy: 0.4793 - val_loss: 1.1476 - val_accuracy: 0.4813 - 39s/epoch - 2s/step
Epoch 4/60
19/19 - 39s - loss: 1.1298 - accuracy: 0.4788 - val_loss: 1.0887 - val_accuracy: 0.4813 - 39s/epoch - 2s/step
Epoch 5/60
19/19 - 39s - loss: 1.0167 - accuracy: 0.5402 - val_loss: 0.9142 - val_accuracy: 0.6361 - 39s/epoch - 2s/step
Epoch 6/60
19/19 - 39s - loss: 0.8626 - accuracy: 0.6643 - val_loss: 0.8738 - val_accuracy: 0.6382 - 39s/epoch - 2s/step
Epoch 7/60
19/19 - 42s - loss: 0.8333 - accuracy: 0.6680 - val_loss: 0.8652 - val_accuracy: 0.6574 - 42s/epoch - 2s/step
Epoch 8/60
19/19 - 40s - loss: 0.7835 - accuracy: 0.6845 - val_loss: 0.7753 - val_accuracy: 0.6681 - 40s/epoch - 2s/step
Epoch 9/60
19/19 - 40s - loss: 0

In [3]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
import random as ra
import time


def train(epochs_num,batch_len,conv_layers,neur_layers):
    X_train,X_test,Y_train,Y_test=train_test_split(
    dataset, labels, test_size=0.2, random_state=32)
    X_train=X_train.reshape(-1,100,100,1)
    X_test=X_test.reshape(-1,100,100,1)
    Y_train = keras.utils.to_categorical(Y_train,3)
    Y_test = keras.utils.to_categorical(Y_test,3)
    
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=2560
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            w=math.sqrt(2.0/inputs)*ra.randint(1,1000)
            hidden=layers.Dense(30,activation="relu",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=24
            w=math.sqrt(2.0/24)*ra.randint(1,1000)
        nl=neur_layers+1
        output=layers.Dense(3,activation="softmax",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/24,maxval=1/24, seed=None))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.2,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [4]:
train(60,200,3,3)

Epoch 1/60
19/19 - 35s - loss: 1.0927 - accuracy: 0.4614 - val_loss: 1.0431 - val_accuracy: 0.5080 - 35s/epoch - 2s/step
Epoch 2/60
19/19 - 27s - loss: 1.0664 - accuracy: 0.4668 - val_loss: 1.0484 - val_accuracy: 0.5080 - 27s/epoch - 1s/step
Epoch 3/60
19/19 - 27s - loss: 1.0619 - accuracy: 0.4668 - val_loss: 1.0508 - val_accuracy: 0.5080 - 27s/epoch - 1s/step
Epoch 4/60
19/19 - 27s - loss: 1.0488 - accuracy: 0.4668 - val_loss: 0.9876 - val_accuracy: 0.5080 - 27s/epoch - 1s/step
Epoch 5/60
19/19 - 27s - loss: 0.9356 - accuracy: 0.4668 - val_loss: 0.8473 - val_accuracy: 0.5080 - 27s/epoch - 1s/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 40)        400       
                                                  

In [41]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,Y,file,aug):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code
    if "virus" in file:
        Y.append("0")
    elif "bacteria" in file:
        Y.append("1")
    else:
        Y.append("2")
    if aug:
        augmentation(X,Y)

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset
    
def create_dataset(directory):
    #before work
    files=glob.glob(directory+'/*.jpeg')
    ra.shuffle(files)
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for file in files[0:int(len(files)*0.8)]:
        process(X_train,Y_train,file,True)
    for file in files[int(len(files)*0.8):]:
        process(X_test,Y_test,file,False)        
    from numpy import asarray
    X_train=normalize(X_train)
    X_test=normalize(X_test)
    return X_train,X_test,Y_train,Y_test

def augmentation(dataset,labels):
    last=len(labels)-1
    scale_out = skimage.transform.rescale(dataset[last], scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in = skimage.transform.rescale(dataset[last], scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    dataset.append(scale_out)
    labels.append(labels[last])
    dataset.append(scale_in)
    labels.append(labels[last])
    

In [18]:
!pip install scikit-image

In [42]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math
X_train,X_test,Y_train,Y_test=create_dataset('chest_xray_lower_dim2')
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train= keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)

In [45]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation="relu",kernel_regularizer=regularizers.L2(0.01),kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=inputs*30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation="softmax",kernel_regularizer=regularizers.L2(0.01))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.2,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [46]:
train(30,200,3,3)

Epoch 1/30
57/57 - 97s - loss: 1.1214 - accuracy: 0.4688 - val_loss: 1.0915 - val_accuracy: 0.4867 - 97s/epoch - 2s/step
Epoch 2/30
57/57 - 92s - loss: 1.0886 - accuracy: 0.4729 - val_loss: 1.0722 - val_accuracy: 0.4867 - 92s/epoch - 2s/step
Epoch 3/30
57/57 - 95s - loss: 1.0787 - accuracy: 0.4729 - val_loss: 1.0626 - val_accuracy: 0.4867 - 95s/epoch - 2s/step
Epoch 4/30


KeyboardInterrupt: 